# HW8 - Neuronal Dynamics (Oscillating Systems)

This week we learned about the FitzHugh Nagumo model of a neuron described by the following equations:

$$\dot{v} = -v(v-a)(v-1)-w $$
$$\dot{w} = \epsilon (v - \gamma w)$$

## Question 1

Solve for the v and w nullclines, define them as functions, and plot them for $\gamma = 2$ and $\gamma = 8$. Use $\epsilon = 0.01$ and $a = 0.1$. Make sure that the X and Y limits of your plots show all relavant dynamics.



## Question 2

Plot the phase portraits for both of the $\gamma$ parameter values. 
How has the phase flow changed by changing $\gamma$? What happens to the set of possible state trajectories when $\gamma$ becomes large?


## Question 3
Numerically integrate the FH equations with the addition of a step impulse.

To add a step impulse to this simulation consider that an outside change in the current $I(t)$ that effect the $\dot{v}$ so that

$$\dot{v} = -v(v-a)(v-1)-w + I(t)$$
$$\dot{w} = \epsilon (v - \gamma w)$$


* Define a new function of the FN equations with a forcing function `input(Ton,q)` where $I(t) = 0$ until `Ton` whereafter $I(t) = q$
* Simulate a sudden increase in I. Plot $v(t)$ and $w(t)$. What happens to the neuron?
* Determine the minimum value of $I$ that causes full spiking to 1 significant digit by trying different input amplitudes. 
* What is the lowest step current amplitude to 1 sig fig to generate **repetitive firing**?

## Question 4 - EXTRA CREDIT (not optional for grads)

Below is a more realistic simulation of a neuron from the famous Hodgkin and Huxley model. While it is more accurate it is less mathematically tractable and thus requires more simulation to get intuition. I have implemented the model with real parameters, ie. they represent real concentrations and voltages.

Here we will use a dictionary to store and alter parameter values. This maintains readability in a rather unwieldy model. Use the following line of code to change the injected voltage:
`params['E_params']['I_ext'] = 1.0e-11`

1) Simulate the neuron so that it only has leakage current and external current. In other words, comment out the terms for sodium and potassium channels. Run a simulation with an initial membrane potential of -70mv and an external current of 0mv. What happens and why?

2) Change the external current to 1e-10 and re-run the simulation. What happens and why?

3) Add back the sodium channel terms (activation and deactivation). Run a simulation with external current 1e-10 and initial states `[-70e-03, 0, 1]`. What happens and why?

4) Add back terms related to the potassium channel. Run a simulation with external current 1e-10 and initial states `[-70e-03, 0, 1, 0]`. What happens and why?

5) What is the effect of external current on neuronal firing behavior? What voltages cause full firing, and repetitive firing. Is there a minimum voltage? Plot the membrane voltage as a function of time to show that this is true. 

In [3]:
# set up a dictionary of parameters so that we can index by name

import numpy as np
from scipy.integrate import odeint
import scipy as sp
import matplotlib.pyplot as plt
import math as m



E_params = {
        'E_leak' : -7.0e-2,
        'G_leak' : 3.0e-09,
        'C_m'    : 3.0e-11,
        'I_ext'  : 1.0e-2
}

Na_params = {
        'Na_E'          : 5.0e-2,
        'Na_G'          : 1.0e-6,
        'k_Na_act'      : 3.0e+0,
        'A_alpha_m_act' : 2.0e+5,
        'B_alpha_m_act' : -4.0e-2,
        'C_alpha_m_act' : 1.0e-3,
        'A_beta_m_act'  : 6.0e+4,
        'B_beta_m_act'  : -4.9e-2,
        'C_beta_m_act'  : 2.0e-2,
        'l_Na_inact'    : 1.0e+0,
        'A_alpha_m_inact' : 8.0e+4,
        'B_alpha_m_inact' : -4.0e-2,
        'C_alpha_m_inact' : 1.0e-3,
        'A_beta_m_inact'  : 4.0e+2,
        'B_beta_m_inact'  : -3.6e-2,
        'C_beta_m_inact'  : 2.0e-3
}

K_params = {
        'k_E'           : -9.0e-2,
        'k_G'           : 2.0e-7,
        'k_K'           : 4.0e+0,
        'A_alpha_m_act' : 2.0e+4,
        'B_alpha_m_act' : -3.1e-2,
        'C_alpha_m_act' : 8.0e-4,
        'A_beta_m_act'  : 5.0e+3,
        'B_beta_m_act'  : -2.8e-2,
        'C_beta_m_act'  : 4.0e-4
}

params = {
        'E_params'  : E_params,
        'Na_params' : Na_params,
        'K_params'  : K_params
}

#Example of how to index the dicts
print(E_params['E_leak'])
print(K_params['k_K'])


def neuron(state, t, params):
        """
         Purpose: simulate Hodgkin and Huxley model for the action potential using
         the equations from Ekeberg et al, Biol Cyb, 1991.
         Input: state ([E m h n] (ie [membrane potential; activation of
                  Na++ channel; inactivation of Na++ channel; activation of K+
                  channel]),
                t (time),
                and the params (parameters of neuron; see Ekeberg et al).
         Output: statep (state derivatives).
        """

        E = state[0]
        m = state[1]
        h = state[2]
        n = state[3]

        Epar = params['E_params']
        Na   = params['Na_params']
        K    = params['K_params']

        # external current (from "voltage clamp", other compartments, other neurons, etc)
        I_ext = Epar['I_ext']

        # calculate Na rate functions and I_Na
        alpha_act = Na['A_alpha_m_act'] * (E-Na['B_alpha_m_act']) / (1.0 - np.exp((Na['B_alpha_m_act']-E) / Na['C_alpha_m_act']))
        beta_act = Na['A_beta_m_act'] * (Na['B_beta_m_act']-E) / (1.0 - np.exp((E-Na['B_beta_m_act']) / Na['C_beta_m_act']) )
        dmdt = ( alpha_act * (1.0 - m) ) - ( beta_act * m )

        alpha_inact = Na['A_alpha_m_inact'] * (Na['B_alpha_m_inact']-E) / (1.0 - np.exp((E-Na['B_alpha_m_inact']) / Na['C_alpha_m_inact']))
        beta_inact  = Na['A_beta_m_inact'] / (1.0 + (np.exp((Na['B_beta_m_inact']-E) / Na['C_beta_m_inact'])))
        dhdt = ( alpha_inact*(1.0 - h) ) - ( beta_inact*h )

        # Na-current:
        I_Na =(Na['Na_E']-E) * Na['Na_G'] * (m**Na['k_Na_act']) * h

        # calculate K rate functions and I_K
        alpha_kal = K['A_alpha_m_act'] * (E-K['B_alpha_m_act']) / (1.0 - np.exp((K['B_alpha_m_act']-E) / K['C_alpha_m_act']))
        beta_kal = K['A_beta_m_act'] * (K['B_beta_m_act']-E) / (1.0 - np.exp((E-K['B_beta_m_act']) / K['C_beta_m_act']))
        dndt = ( alpha_kal*(1.0 - n) ) - ( beta_kal*n )
        I_K = (K['k_E']-E) * K['k_G'] * n**K['k_K']

        # leak current
        I_leak = (Epar['E_leak']-E) * Epar['G_leak']

        # calculate derivative of E
        dEdt = (I_leak + I_K + I_Na + I_ext) / Epar['C_m']
        statep = [dEdt, dmdt, dhdt, dndt]

        return statep

-0.07
4.0


In [14]:

# set initial states and time vector
state0 = [-70e-03, 0, 1, 0]
t = np.arange(0, 0.2, 0.001)

# let's inject some external current
#params['E_params']['I_ext'] = #Enter parameter value here

# run simulation
state = odeint(neuron, state0, t, args=(params,))

#### Who did you work with?
